# Step 1 - Create DataFrame

First, the data needs to be imported for mining and manipulation. The data is fetched from CSV file and presented as Dask DataFrame.

In [40]:
import dask.dataframe as pd
import pandas as px
import numpy as np
from astral import Astral

astral = Astral()
city = astral['London']

df = pd.read_csv("data/travistorrent_11_1_2017.csv",
                 usecols=['tr_build_id', 'tr_prev_build', 'tr_status', 'gh_build_started_at', 'gh_project_name', 'git_branch'],
                parse_dates=['gh_build_started_at'])
df = df.drop_duplicates()
moon = map(lambda x: city.moon_phase(date=x), df['gh_build_started_at'].compute())
df = df.compute().assign(moon_phase = list(moon))

Combine with "previous build", add transition as 'status_update' (old:new), 'delta' between start times of the builds and create frame containing only builds where status changed.

In [41]:
df_follower = df[(df['tr_prev_build'] != np.nan) & (df['tr_prev_build'] > 0)]
df_with_prev = pd.merge(df_follower, df, left_on='tr_prev_build', right_on='tr_build_id', suffixes=('', '_prev'))
df_with_prev = df_with_prev.assign(delta = df_with_prev['gh_build_started_at'] - df_with_prev['gh_build_started_at_prev'])
df_with_prev = df_with_prev.assign(status_update = df_with_prev['tr_status_prev'] + ':' + df_with_prev['tr_status'])
df_with_change = df_with_prev[df_with_prev['tr_status'] != df_with_prev['tr_status_prev']]

There seem to be some previous builds that started in the future...

In [42]:
from datetime import timedelta
df_with_prev[df_with_prev['delta'] < timedelta(0)].sort_values('delta')

,tr_build_id,gh_project_name,git_branch,tr_prev_build,gh_build_started_at,tr_status,moon_phase,tr_build_id_prev,gh_project_name_prev,git_branch_prev,tr_prev_build_prev,gh_build_started_at_prev,tr_status_prev,moon_phase_prev,delta,status_update
271760,35739610,caelum/tubaina,master,35656796.0,2014-09-19 15:05:12,errored,24,35656796,caelum/tubaina,master,32007968.0,2016-02-04 18:53:09,errored,23,-504 days +20:12:03,errored:errored
182881,22925036,topfunky/gruff,master,17244194.0,2014-04-14 05:08:16,passed,13,17244194,topfunky/gruff,master,17242052.0,2015-05-30 22:17:42,passed,11,-412 days +06:50:34,passed:passed
182882,23664005,topfunky/gruff,master,17244194.0,2014-04-24 10:31:01,passed,23,17244194,topfunky/gruff,master,17242052.0,2015-05-30 22:17:42,passed,11,-402 days +12:13:19,passed:passed
182883,27314542,topfunky/gruff,master,17244194.0,2014-06-11 15:04:55,errored,12,17244194,topfunky/gruff,master,17242052.0,2015-05-30 22:17:42,passed,11,-354 days +16:47:13,passed:errored
237474,28225579,topfunky/gruff,master,27353384.0,2014-06-23 13:57:57,errored,24,27353384,topfunky/gruff,master,NaN,2015-05-30 21:59:11,failed,11,-342 days +15:58:46,failed:errored
72916,4734455,codebrew/backbone-rails,master,4536321.0,2013-02-11 23:51:45,passed,1,4536321,codebrew/backbone-rails,master,4536018.0,2014-01-17 22:54:25,failed,15,-340 days +00:57:20,failed:passed
238616,28443919,topfunky/gruff,master,28292510.0,2014-06-25 20:25:55,errored,26,28292510,topfunky/gruff,master,NaN,2015-05-30 21:55:13,failed,11,-340 days +22:30:42,failed:errored
294141,40834277,jekyll/mercenary,require-absolute,40832893.0,2014-11-12 23:24:23,passed,19,40832893,jekyll/mercenary,require-absolute,40821425.0,2015-06-17 15:44:02,errored,0,-217 days +07:40:21,errored:passed
294142,40834281,jekyll/mercenary,master,40832893.0,2014-11-12 23:24:50,canceled,19,40832893,jekyll/mercenary,require-absolute,40821425.0,2015-06-17 15:44:02,errored,0,-217 days +07:40:48,errored:canceled
241244,28969435,DozerMapper/dozer,master,23574010.0,2014-07-02 14:39:45,passed,4,23574010,DozerMapper/dozer,master,23534488.0,2014-11-23 10:54:31,passed,0,-144 days +03:45:14,passed:passed


... some of the builds where run on different branches ...

In [43]:
df_with_prev[df_with_prev['git_branch'] != df_with_prev['git_branch_prev']]

,tr_build_id,gh_project_name,git_branch,tr_prev_build,gh_build_started_at,tr_status,moon_phase,tr_build_id_prev,gh_project_name_prev,git_branch_prev,tr_prev_build_prev,gh_build_started_at_prev,tr_status_prev,moon_phase_prev,delta,status_update
14,108486,karmi/retire,proxy,108310.0,2011-08-29 12:51:08,passed,0,108310,karmi/retire,master,105901.0,2011-08-29 12:42:29,passed,0,0 days 00:08:39,passed:passed
153,117786,nov/fb_graph,ads,117598.0,2011-09-01 04:11:43,failed,3,117598,nov/fb_graph,master,NaN,2011-09-01 04:04:21,failed,3,0 days 00:07:22,failed:failed
203,121379,thoughtbot/factory_girl,rails-3-1,120009.0,2011-09-02 15:04:20,passed,4,120009,thoughtbot/factory_girl,master,115162.0,2011-09-01 22:23:00,passed,3,0 days 16:41:20,passed:passed
205,121386,tscolari/mobylette,view_helpers,120215.0,2011-09-02 13:53:39,failed,4,120215,tscolari/mobylette,helper_tests,NaN,2011-09-02 02:26:46,failed,4,0 days 11:26:53,failed:failed
213,121807,tscolari/mobylette,master,121720.0,2011-09-02 16:28:16,failed,4,121720,tscolari/mobylette,view_helpers,121386.0,2011-09-02 16:24:45,failed,4,0 days 00:03:31,failed:failed
263,124317,padrino/padrino-framework,minitest,124247.0,2011-09-03 11:56:50,failed,5,124247,padrino/padrino-framework,master,123042.0,2011-09-03 11:07:38,errored,5,0 days 00:49:12,errored:failed
281,4727872,flori/json,fix-additions-problem-v1.5.5,113436.0,2013-02-11 18:55:01,errored,1,113436,flori/json,master,109977.0,2011-08-31 00:26:40,passed,2,530 days 18:28:21,passed:errored
415,130327,afeld/magickly,synchrony,130021.0,2011-09-05 08:19:43,passed,7,130021,afeld/magickly,master,90537.0,2011-09-05 07:12:33,passed,7,0 days 01:07:10,passed:passed
419,130476,guard/guard,jruby_specs,130211.0,2011-09-05 08:50:08,passed,7,130211,guard/guard,master,NaN,2011-09-05 08:19:53,failed,7,0 days 00:30:15,failed:passed
423,131888,afeld/magickly,cedar,130341.0,2011-09-05 19:10:24,passed,7,130341,afeld/magickly,master,130021.0,2011-09-05 08:35:11,passed,7,0 days 10:35:13,passed:passed


... and there are some 'previous builds' missing.

In [44]:
prev_ids = set(df_follower['tr_prev_build'])
build_ids = set(df['tr_build_id'])
unref_prev = prev_ids - build_ids
len(unref_prev)

109

And now for some statistics:

Which status transitions occur?

In [45]:
df_with_prev.groupby('status_update').count().sort_values('tr_build_id')

,tr_build_id,gh_project_name,git_branch,tr_prev_build,gh_build_started_at,tr_status,moon_phase,tr_build_id_prev,gh_project_name_prev,git_branch_prev,tr_prev_build_prev,gh_build_started_at_prev,tr_status_prev,moon_phase_prev,delta
status_update,,,,,,,,,,,,,,,
started:passed,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
passed:started,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
canceled:errored,236,236,236,236,236,236,236,236,236,236,207,236,236,236,236
errored:canceled,264,264,264,264,264,264,264,264,264,264,226,264,264,264,264
canceled:failed,396,396,396,396,396,396,396,396,396,396,340,396,396,396,396
canceled:canceled,443,443,443,443,443,443,443,443,443,443,393,443,443,443,443
failed:canceled,491,491,491,491,491,491,491,491,491,491,418,491,491,491,491
passed:canceled,743,743,743,743,743,743,743,743,743,743,543,743,743,743,743
canceled:passed,893,893,893,893,893,893,893,893,893,893,746,893,893,893,893
